# Data Analysis

In [1]:
import os
import pandas as pd
import json
import datetime as dt
import collections
import intervaltree

In [2]:
data_dir = os.path.expanduser("~/data/flare-benchmark/")
raw_dir = os.path.join(data_dir, "raw")

events_file = os.path.join(raw_dir, "events_2012-01-01T00:00:00_2018-01-01T00:00:00.json")

In [3]:
with open(events_file, "r") as f:
    events = json.load(f)

In [4]:
def dtf(raw_date):
    return dt.datetime.strptime(raw_date, "%Y-%m-%dT%H:%M:%S")

swpc_flares = list(sorted(
    (event for event in events if event["event_type"] == "FL" and event["frm_name"] == "SWPC"),
    key=lambda event: (dtf(event["event_starttime"]), dtf(event["event_peaktime"]), dtf(event["event_endtime"]))
))
print("SWPC flares:", len(swpc_flares))

spoca_regions = list(sorted(
    (event for event in events if event["event_type"] == "AR" and event["frm_name"] == "SPoCA"),
    key=lambda event: (dtf(event["event_starttime"]), dtf(event["event_endtime"]))
))

print("SPoCA regions:", len(spoca_regions))

spoca_regions_grouped = dict()
for region in spoca_regions:
    current_id = region["frm_specificid"]
    if current_id not in spoca_regions_grouped:
        spoca_regions_grouped[current_id] = [region]
    else:
        spoca_regions_grouped[current_id].append(region)

print("SPoCA regions (grouped):", len(spoca_regions_grouped))

SWPC flares: 10954
SPoCA regions: 76983
SPoCA regions (grouped): 11246


In [5]:
noaa_regions = dict()

for event in events:
    if event["event_type"] == "AR" and event["frm_name"] == "NOAA SWPC Observer":
        noaa_id = event["ar_noaanum"]
        
        if noaa_id not in noaa_regions:
            noaa_regions[noaa_id] = [event]
        else:
            noaa_regions[noaa_id].append(event)

test_region = noaa_regions[11482]
for event in test_region:
    print(event["ar_noaanum"], event["event_starttime"], event["event_endtime"])
    
print()

num_fl_with_ar = 0
num_fl_without_ar = 0
for event in swpc_flares:
    if event["ar_noaanum"] > 0:
        num_fl_with_ar += 1
    else:
        num_fl_without_ar += 1
print(num_fl_with_ar, num_fl_without_ar)

print()

has_flare = {noaa_id: False for noaa_id in noaa_regions.keys()}

for event in swpc_flares:
    if event["ar_noaanum"] > 0:
        has_flare[event["ar_noaanum"]] = True

num_regions_with_fl = sum(has_flare.values())
num_regions_without_fl = sum(not c for c in has_flare.values())

print(len(noaa_regions), num_regions_with_fl, num_regions_without_fl)


11482 2012-05-14T00:00:00 2012-05-14T23:59:59
11482 2012-05-15T00:00:00 2012-05-15T23:59:59
11482 2012-05-16T00:00:00 2012-05-16T23:59:59
11482 2012-05-17T00:00:00 2012-05-17T23:59:59
11482 2012-05-18T00:00:00 2012-05-18T23:59:59
11482 2012-05-19T00:00:00 2012-05-19T23:59:59
11482 2012-05-20T00:00:00 2012-05-20T23:59:59
11482 2012-05-21T00:00:00 2012-05-21T23:59:59
11482 2012-05-22T00:00:00 2012-05-22T23:59:59
11482 2012-05-23T00:00:00 2012-05-23T23:59:59
11482 2012-05-24T00:00:00 2012-05-24T23:59:59

9314 1640

1304 837 468


# Data Pipeline

## Step 1: Load NOAA Active Regions and SWPC Flares

In [6]:
noaa_regions = dict()
for event in events:
    if event["event_type"] == "AR" and event["frm_name"] == "NOAA SWPC Observer":
        noaa_id = event["ar_noaanum"]
        
        if noaa_id not in noaa_regions:
            noaa_regions[noaa_id] = [event]
        else:
            noaa_regions[noaa_id].append(event)

noaa_regions = {
    noaa_id: (
        min(map(lambda event: dtf(event["event_starttime"]), events)),
        max(map(lambda event: dtf(event["event_endtime"]), events)),
        events
    )
    for noaa_id, events in noaa_regions.items()
}

swpc_flares = list(sorted(
    (event for event in events if event["event_type"] == "FL" and event["frm_name"] == "SWPC"),
    key=lambda event: (dtf(event["event_starttime"]), dtf(event["event_peaktime"]), dtf(event["event_endtime"]))
))

## Step 2: Assign SWPC Flares to NOAA Active Regions

### Step 2.1: Assign Flares containing Active Region Number

In [7]:
mapped_flares = []
unmapped_flares = []
for event in swpc_flares:
    region_number = event["ar_noaanum"]
    
    if region_number > 0:
        if region_number in noaa_regions.keys():
            mapped_flares.append((event, region_number))
        else:
            print(f"NOAA number {region_number} not present")
            unmapped_flares.append(event)
    else:
        assert region_number == 0
        unmapped_flares.append(event)

print("Mapped flares:", len(mapped_flares))
print("Unmapped flares:", len(unmapped_flares))

NOAA number 12689 not present
NOAA number 12689 not present
Mapped flares: 9312
Unmapped flares: 1642


### Step 2.2: Assign remaining Flares using Heuristics (SSW Events)

In [8]:
ssw_flares = filter(lambda event: event["event_type"] == "FL" and event["frm_name"] == "SSW Latest Events", events)
ssw_flares = [(
    dtf(event["event_starttime"]),
    dtf(event["event_peaktime"]),
    dtf(event["event_endtime"]),
    event
    ) for event in ssw_flares
]

still_unmapped_flares = []

for event in unmapped_flares:
    event_start = dtf(event["event_starttime"])
    event_end = dtf(event["event_endtime"])
    event_peak = dtf(event["event_peaktime"])
    
    # Find matching SSW event
    match_start, match_peak, match_end, match_event = min(
        ssw_flares,
        key=lambda cand: (
            abs(event_peak - cand[1]),
            abs(event_end - cand[2]),
            abs(event_start - cand[0])
        )
    )
    
    # Check peak time delta
    if abs(event_peak - match_peak) > dt.timedelta(minutes=1):
        if event_start != match_start or event_end != match_end or abs(event_peak - match_peak) > dt.timedelta(minutes=10):
            still_unmapped_flares.append(event)
            continue
    
    # Check if event peak is inside match duration
    if not (match_start <= event_peak <= match_end):
        still_unmapped_flares.append(event)
        continue

    # Times are equal, check if NOAA id is found
    region_number = match_event["ar_noaanum"]

    if region_number == 0:
        still_unmapped_flares.append(event)
        continue
    
    if event["fl_goescls"] != match_event["fl_goescls"]:
        still_unmapped_flares.append(event)
        continue

    assert region_number < 10000
    region_number += 10000
    if region_number in noaa_regions.keys():
        mapped_flares.append((event, region_number))
    else:
        print(f"NOAA number {region_number} not present")
        still_unmapped_flares.append(event)

print("Mapped flares:", len(mapped_flares))
print("Still unmapped flares:", len(still_unmapped_flares))

NOAA number 11383 not present
NOAA number 12689 not present
NOAA number 12689 not present
Mapped flares: 10140
Still unmapped flares: 814


### Step 3: Divide Active Region Time Frames

In [9]:
unmapped_ranges = intervaltree.IntervalTree()
for event in still_unmapped_flares:
    unmapped_start = dtf(event["event_starttime"])
    unmapped_end = dtf(event["event_endtime"])
    unmapped_ranges.addi(unmapped_start, unmapped_end)
unmapped_ranges.merge_overlaps()

DELTA_IN = dt.timedelta(hours=12)
DELTA_OUT = dt.timedelta(hours=12)

# TODO: Check edge-case handling (interval end is often inclusive but tree treats it as exclusive)

ranges = dict()
for noaa_id, (region_start, region_end, region_events) in noaa_regions.items():
    flares = list(sorted(
        (flare_event for flare_event, flare_region_id in mapped_flares if flare_region_id == noaa_id),
        key=lambda flare_event: dtf(flare_event["event_peaktime"])
    ))
    
    free_ranges = intervaltree.IntervalTree()
    free_ranges.addi(region_start, region_end, "free")
    
    flare_ranges = intervaltree.IntervalTree()
    for idx, flare_event in enumerate(flares):
        flare_peak = dtf(flare_event["event_peaktime"])
        flare_class = flare_event["fl_goescls"]
        
        # Calculate best case range
        range_min = max(flare_peak - DELTA_OUT, region_start + DELTA_IN)
        range_max = min(flare_peak + DELTA_OUT, region_end)
        
        # Check if any bigger flares happen before
        prev_idx = idx - 1
        while prev_idx >= 0 and dtf(flares[prev_idx]["event_peaktime"]) > range_min:
            # Check if check flare is bigger than current, if yes, reduce start range
            if flares[prev_idx]["fl_goescls"] > flare_class:
                range_min = dtf(flares[prev_idx]["event_peaktime"])
                break
            else:
                prev_idx -= 1

        # Check if any bigger flares happen afterwards
        next_idx = idx + 1
        while next_idx < len(flares) and dtf(flares[next_idx]["event_peaktime"]) < range_max:
            # Check if check flare is bigger than current, if yes, reduce start range
            if flares[next_idx]["fl_goescls"] > flare_class:
                range_max = dtf(flares[next_idx]["event_peaktime"])
                break
            else:
                next_idx += 1
        
        if range_max - range_min >= DELTA_OUT:
            assert range_min <= flare_peak <= range_max
            assert region_start <= flare_peak <= region_end
            assert range_min < range_max
            assert range_min - DELTA_IN >= region_start
            
            flare_ranges.addi(range_min, range_max, flare_class)

        # Remove range around flare from free areas
        free_ranges.chop(flare_peak - DELTA_OUT, flare_peak + DELTA_OUT)

    # Merge free and flare ranges
    region_ranges = free_ranges | flare_ranges
    
    # Remove unmapped ranges from result
    for current_chop_interval in unmapped_ranges:
        region_ranges.chop(*current_chop_interval)
        
        if len(region_ranges[current_chop_interval]) > 0:
            # This seems to be a bug in the library where the chop operation does nothing
            # Creating a new tree seems to fix it
            region_ranges = intervaltree.IntervalTree(set(region_ranges))
            region_ranges.chop(*current_chop_interval)
        
        assert len(region_ranges[current_chop_interval]) == 0
    
    # Remove ranges which are too small
    intervals_remove = [interval for interval in region_ranges if interval.end - interval.begin < DELTA_OUT]
    for current_interval in intervals_remove:
        assert current_interval in region_ranges
        region_ranges.discardi(*current_interval)
        assert current_interval not in region_ranges

    ranges[noaa_id] = region_ranges


In [10]:
for noaa_id, region_ranges in ranges.items():
    # Check length is valid
    assert not any(interval.end - interval.begin < DELTA_OUT for interval in region_ranges)
    
    # Check no overlaps between free regions
    sorted_intervals = list(sorted(interval for interval in region_ranges if interval.data == "free"))
    for idx, interval in enumerate(sorted_intervals):
        # Check free regions within themselfs
        assert idx == 0 or interval.begin > sorted_intervals[idx - 1].end, \
            f"Found overlap between free intervals {interval} and {sorted_intervals[idx - 1]}"
        
        # Check free regions and flares
        assert len(region_ranges[interval]) == 1
    
    # Check distance between flare ranges
    sorted_flare_intervals = list(sorted(interval for interval in region_ranges if interval.data != "free"))
    for idx, interval in enumerate(sorted_flare_intervals):
        assert idx == 0 or interval.begin + DELTA_OUT >= sorted_flare_intervals[idx - 1].end or interval.data <= sorted_flare_intervals[idx - 1].data, \
            f"Found overlap between flares {interval} and {sorted_flare_intervals[idx - 1]}"

    # Check no unmapped flares are overlapped
    for interval in region_ranges:
        assert len(unmapped_ranges[interval]) == 0, \
            f"{interval} overlaps unmapped flare ranges {unmapped_ranges[interval]}"

### Step 4: Verification

In [11]:
# TODO: Some active regions might still produce flares which are not detected by SWPC
# TODO: Make sure no instrument issues are present (e.g. satellite maneuvers)
# TODO: Could also use SPoCA ARs to get way more data
# TODO: Check if active regions overlap each other to avoid duplicates
# TODO: Check if SSW data is actually reliable
# TODO: Can we really use arbitrary data if DELTA_IN > 0? (Except for checking if the complete AR is visible)

### Step 5: Evaluation

In [12]:
num_ranges_classes = collections.defaultdict(int)

for region_ranges in ranges.values():
    for current_range in region_ranges:
        current_class = "fl" if current_range.data != "free" else current_range.data
        
        num_ranges_classes[current_class] += 1

num_ranges_classes

defaultdict(int, {'fl': 5395, 'free': 2783})